In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor
import requests as r
import pandas_datareader as pdr
from dotenv import load_dotenv
import os
load_dotenv()
token = os.getenv("ALPHA")
from time import sleep

In [2]:
market = Market()
sp500 = ADatabase("sp500")
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
market.connect()
start = market.retrieve_max_date("prices")
market.disconnect()
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2022-10-11 2022-10-12


In [4]:
market.connect()
for ticker in tqdm(list(sp5["Symbol"].unique())):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        try:
            data = TiingoExtractor.extract(ticker,start,end)
            data["ticker"] = ticker
            market.store("prices",data)
        except:
            print(ticker,"tiingo")
        try:
            test = pdr.yahoo.daily.YahooDailyReader(symbols=ticker
                                                    , start=start
                                                    , end=end)
            panda_reader_data = test.read()
            panda_reader_data["ticker"] = ticker
            panda_reader_data.reset_index(inplace=True)
            market.store("pdr_prices",panda_reader_data)
        except:
            print(ticker,"pdr")
        sleep(1)
    except Exception as e:
        print(str(e))
market.disconnect()

 17%|█████████████████████████▍                                                                                                                              | 83/497 [03:30<17:07,  2.48s/it]

COF pdr


 22%|█████████████████████████████████▋                                                                                                                     | 111/497 [05:11<16:34,  2.58s/it]

market prices documents must be a non-empty list
CTXS pdr


 32%|███████████████████████████████████████████████▋                                                                                                       | 157/497 [07:08<14:11,  2.50s/it]

...

 56%|████████████████████████████████████████████████████████████████████████████████████▊                                                                  | 279/497 [15:40<15:58,  4.40s/it]

LNC pdr


 88%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                  | 437/497 [25:34<04:11,  4.19s/it]

TXN pdr


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 497/497 [30:22<00:00,  3.67s/it]


In [5]:
analysis = []
market.connect()
periods = ["year","quarter","month","week"]
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("prices",ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 497/497 [00:16<00:00, 30.27it/s]


In [6]:
market.connect()
try:
    try:
        test = pdr.yahoo.daily.YahooDailyReader(symbols="BTC-USD", start=start, end=end)
        panda_reader_data = test.read()
        panda_reader_data["ticker"] = "BTC"
        panda_reader_data.reset_index(inplace=True)
        market.store("crypto",panda_reader_data)
    except:
        print("btc","pdr")
except Exception as e:
    print(str(e))
market.disconnect()

In [7]:
period = "day"
a = pd.DataFrame(analysis)
a = a.merge(sp5[["Symbol","Security","GICS Sector","GICS Sub-Industry"]].rename(columns={"Symbol":"ticker"}),on="ticker")
a.sort_values(f"{period}_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector,GICS Sub-Industry
467,-0.318826,0.076835,0.076835,0.073143,0.073143,VTRS,Viatris,Health Care,Pharmaceuticals
38,0.109288,0.065093,0.065093,0.057247,0.057247,AMGN,Amgen,Health Care,Biotechnology
308,-0.437190,0.038476,0.038476,0.045348,0.045348,MU,Micron Technology,Information Technology,Semiconductors
86,-0.689865,-0.030657,-0.030657,0.040752,0.040752,CCL,Carnival,Consumer Discretionary,"Hotels, Resorts & Cruise Lines"
271,0.025037,0.044114,0.044114,0.032083,0.032083,KR,Kroger,Consumer Staples,Food Retail
36,0.078559,0.017228,0.017228,0.029720,0.029720,ABC,AmerisourceBergen,Health Care,Health Care Distributors
381,-0.553851,0.029493,0.029493,0.029274,0.029274,PVH,PVH,Consumer Discretionary,"Apparel, Accessories & Luxury Goods"
222,-0.366665,-0.065657,-0.065657,0.027302,0.027302,PEAK,Healthpeak,Real Estate,Health Care REITs
441,-0.136135,0.034206,0.034206,0.026223,0.026223,TSCO,Tractor Supply,Consumer Discretionary,Specialty Stores
453,-0.664783,0.012802,0.012802,0.025937,0.025937,UAA,Under Armour (Class A),Consumer Discretionary,"Apparel, Accessories & Luxury Goods"


In [8]:
a.groupby("GICS Sector").mean().sort_values("quarter_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Energy,0.405987,0.044011,0.044011,-0.008261,-0.008261
Consumer Staples,-0.062144,0.002648,0.002648,0.008900,0.008900
Consumer Discretionary,-0.311367,-0.000487,-0.000487,-0.001599,-0.001599
Industrials,-0.172929,-0.002180,-0.002180,-0.001420,-0.001420
Health Care,-0.180141,-0.011871,-0.011871,0.002781,0.002781
Materials,-0.179672,-0.015377,-0.015377,-0.007794,-0.007794
Financials,-0.183717,-0.017697,-0.017697,-0.010300,-0.010300
Communication Services,-0.318124,-0.031280,-0.031280,-0.016517,-0.016517
Information Technology,-0.312841,-0.048026,-0.048026,-0.019137,-0.019137
